In [1]:
import torch
import numpy as np
import hashlib
from datetime import datetime

class ImprovedKeyGenerator:
    """Enhanced chaotic key generation with better properties"""

    def __init__(self, seed=None):
        if seed is None:
            seed = int(datetime.now().timestamp() * 1000000) % (2**32)
        self.seed = seed
        np.random.seed(seed)

    def exponential_chebyshev_map(self, x, mu=2.5):
        """1-DEC map from paper - better than logistic map"""
        return 1 - 2 * (np.cos(np.arccos(x) * np.exp(abs(mu) * np.arccos(x))))**2

    def tent_map(self, x, mu=1.9):
        """Additional chaotic map for mixing"""
        if x < 0.5:
            return mu * x
        else:
            return mu * (1 - x)

    def generate_key_sequence(self, length, initial, map_type='chebyshev'):
        """Generate chaotic sequence with specified map"""
        sequence = []
        x = initial

        for _ in range(length):
            if map_type == 'chebyshev':
                x = self.exponential_chebyshev_map(x)
            else:
                x = self.tent_map(x)
            sequence.append(x)

        return np.array(sequence, dtype=np.float32)

    def generate_multiple_keys(self, num_keys=8, length=512):
        """Generate 8 independent key sequences (matching paper)"""
        keys = []

        for i in range(num_keys):
            # Use different initial conditions
            initial = 0.25454 + (i * 0.123456) / num_keys
            initial = initial % 1.0

            # Alternate between map types for diversity
            map_type = 'chebyshev' if i % 2 == 0 else 'tent'
            key_seq = self.generate_key_sequence(length, initial, map_type)

            keys.append(key_seq)

        return keys


class StringProcessor:
    """String to integer conversion (printable ASCII)"""

    def __init__(self, max_length=128):
        self.max_length = max_length
        self.char_to_idx = {}
        self.idx_to_char = {}

        self.PAD = 0
        self.START = 1
        self.END = 2

        idx = 3
        for ascii_val in range(32, 127):
            char = chr(ascii_val)
            self.char_to_idx[char] = idx
            self.idx_to_char[idx] = char
            idx += 1

        self.vocab_size = idx

    def encode(self, text):
        if len(text) > self.max_length - 2:
            text = text[:self.max_length - 2]

        indices = [self.START]
        for char in text:
            if char in self.char_to_idx:
                indices.append(self.char_to_idx[char])
            else:
                indices.append(self.char_to_idx[' '])
        indices.append(self.END)

        while len(indices) < self.max_length:
            indices.append(self.PAD)

        return torch.LongTensor(indices)

    def decode(self, indices):
        if isinstance(indices, torch.Tensor):
            indices = indices.cpu().numpy()

        chars = []
        for idx in indices:
            idx = int(idx)
            if idx == self.START:
                continue
            elif idx == self.END:
                break
            elif idx == self.PAD:
                continue
            elif idx in self.idx_to_char:
                chars.append(self.idx_to_char[idx])

        return ''.join(chars)

    def batch_encode(self, texts):
        return torch.stack([self.encode(text) for text in texts])

    def batch_decode(self, indices_batch):
        return [self.decode(indices) for indices in indices_batch]


class KeyManager:
    """Manages key generation with message-dependent seeding"""

    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.key_gen = ImprovedKeyGenerator()

    def generate_keys_for_batch(self, batch_size, seq_length, message_hash=None):
        """Generate keys with optional message-dependent seed"""

        if message_hash is not None:
            seed = int(hashlib.sha256(message_hash.encode()).hexdigest()[:8], 16)
            self.key_gen = ImprovedKeyGenerator(seed=seed)

        raw_keys = self.key_gen.generate_multiple_keys(num_keys=8, length=seq_length)

        key_tensors = []
        for key in raw_keys:
            key_tensor = torch.FloatTensor(key).to(self.device)
            key_tensor = key_tensor.unsqueeze(0).expand(batch_size, -1)
            key_tensors.append(key_tensor)

        return {
            'raw_keys': raw_keys,
            'key_tensors': key_tensors,
            'num_keys': len(raw_keys)
        }


if __name__ == "__main__":
    print("="*70)
    print("IMPROVED KEY GENERATOR V3")
    print("="*70)

    processor = StringProcessor(max_length=128)
    key_manager = KeyManager()

    test_msg = "Hello, World! Testing 123."
    print(f"\nOriginal: '{test_msg}'")

    encoded = processor.encode(test_msg)
    decoded = processor.decode(encoded)
    print(f"Decoded: '{decoded}'")
    print(f"Match: {decoded == test_msg}")

    batch_size = 4
    seq_len = 128
    keys = key_manager.generate_keys_for_batch(batch_size, seq_len)

    print(f"\nGenerated {keys['num_keys']} key sequences")
    print(f"Each key shape: {keys['key_tensors'][0].shape}")
    print(f"Key 0 sample: {keys['key_tensors'][0][0, :10]}")

    print("\n" + "="*70)

IMPROVED KEY GENERATOR V3

Original: 'Hello, World! Testing 123.'
Decoded: 'Hello, World! Testing 123.'
Match: True

Generated 8 key sequences
Each key shape: torch.Size([4, 128])
Key 0 sample: tensor([-0.5861, -0.7212,  0.8895,  0.9994, -0.9974, -0.2422, -0.9082, -0.4629,
         0.8816,  0.9762])

